# Assignment 2

Before working on this assignment please read these instructions fully. In the submission area, you will notice that you can click the link to **Preview the Grading** for each step of the assignment. This is the criteria that will be used for peer grading. Please familiarize yourself with the criteria before beginning the assignment.

An NOAA dataset has been stored in the file `data/C2A2_data/BinnedCsvs_d400/fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89.csv`. This is the dataset to use for this assignment. Note: The data for this assignment comes from a subset of The National Centers for Environmental Information (NCEI) [Daily Global Historical Climatology Network](https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/readme.txt) (GHCN-Daily). The GHCN-Daily is comprised of daily climate records from thousands of land surface stations across the globe.

Each row in the assignment datafile corresponds to a single observation.

The following variables are provided to you:

* **id** : station identification code
* **date** : date in YYYY-MM-DD format (e.g. 2012-01-24 = January 24, 2012)
* **element** : indicator of element type
    * TMAX : Maximum temperature (tenths of degrees C)
    * TMIN : Minimum temperature (tenths of degrees C)
* **value** : data value for element (tenths of degrees C)

For this assignment, you must:

1. Read the documentation and familiarize yourself with the dataset, then write some python code which returns a line graph of the record high and record low temperatures by day of the year over the period 2005-2014. The area between the record high and record low temperatures for each day should be shaded.
2. Overlay a scatter of the 2015 data for any points (highs and lows) for which the ten year record (2005-2014) record high or record low was broken in 2015.
3. Watch out for leap days (i.e. February 29th), it is reasonable to remove these points from the dataset for the purpose of this visualization.
4. Make the visual nice! Leverage principles from the first module in this course when developing your solution. Consider issues such as legends, labels, and chart junk.

The data you have been given is near **Ann Arbor, Michigan, United States**, and the stations the data comes from are shown on the map below.

In [1]:
%matplotlib notebook
import matplotlib.pyplot as plt
import mplleaflet
import pandas as pd

def leaflet_plot_stations(binsize, hashid):

    df = pd.read_csv('data/C2A2_data/BinSize_d{}.csv'.format(binsize))

    station_locations_by_hash = df[df['hash'] == hashid]

    lons = station_locations_by_hash['LONGITUDE'].tolist()
    lats = station_locations_by_hash['LATITUDE'].tolist()

    plt.figure(figsize=(8,8))

    plt.scatter(lons, lats, c='r', alpha=0.7, s=200)

    return mplleaflet.display()

leaflet_plot_stations(400,'fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89')

ModuleNotFoundError: No module named 'mplleaflet'

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
import mplleaflet
import pandas as pd
import numpy as np
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Import the data
df = pd.read_csv('data/C2A2_data/BinnedCsvs_d400/fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89.csv')
df.head()

In [ ]:
#Pre_process data for analyzing
#Temperature to tenths
df['Data_Value'] = df['Data_Value'] * 0.1

#Extract year and month-day from 'Date' column
df['Date'] = pd.to_datetime(df['Date'])
df['Year'] = df['Date'].dt.year
df['MonthDay'] = df['Date'].dt.strftime('%m-%d') 

#Remove the leap year dates
df = df[df['MonthDay'] != '02-29']

In [ ]:
#Remove 2015 data from df. Place into new dataframe
data_2015 = df[df['Year']==2015]

#Create new dataframe without 2015
data_2005_2014 = df[df['Year']!=2015]

In [ ]:
# Process: Group data to create series objects that determine max and min temperatures for 2015 and 2005-2014 dataframe

# Create maximum and minimum dataframes with maximum and minimum values over a 10 year period. 
#  Max and Min for 2005-2014 with reset inndex
max_data_2005_2014 = data_2005_2014.groupby('MonthDay').agg({'Data_Value':np.max})
min_data_2005_2014 = data_2005_2014.groupby('MonthDay').agg({'Data_Value':np.min})
max_data_2005_2014.reset_index(inplace=True)
min_data_2005_2014.reset_index(inplace=True)

# Max and Min for 2015
max_data_2015 = data_2015.groupby('MonthDay').agg({'Data_Value':np.max})
min_data_2015 = data_2015.groupby('MonthDay').agg({'Data_Value':np.min})
max_data_2015.reset_index(inplace=True)
min_data_2015.reset_index(inplace=True)

# Merge data from 2015 and 2005-2014 to get complete aggregated data dataframe
all_data_max = pd.merge(max_data_2015, max_data_2005_2014, how='right', on = 'MonthDay').rename(columns = {'Data_Value_x': '2015 Max Temp', 'Data_Value_y': '05-14 Max Temp'})
all_data_min = pd.merge(min_data_2015, min_data_2005_2014, how='right', on = 'MonthDay').rename(columns ={'Data_Value_x': '2015 Min Temp', 'Data_Value_y': '05-14 Min Temp'})

# Dataframes of 2015 data which had points greater than and less than 10 year records (2005-2014). This is for scatter overlay.
max_record_2015 = all_data_max[all_data_max['2015 Max Temp'] > all_data_max['05-14 Max Temp']]
min_record_2015 = all_data_min[all_data_min['2015 Min Temp'] < all_data_min['05-14 Min Temp']]


In [ ]:
%matplotlib inline

#Create figure object with assumed figsize 
plt.figure(figsize = [16,10])

#Plot the 2005-2014 data with 10 year record max and min temperatures for each day in the year
#Create observation points because plot does not take string data
XAxisDates= list(range(1,366))
plt.plot(XAxisDates, max_data_2005_2014['Data_Value'].values, 'b-', label = "Max Temperature")
plt.plot(XAxisDates, min_data_2005_2014['Data_Value'].values, 'g-', label = "Min Temperature")

#Label axes
axes = plt.gca()
axes.set_ylabel('Temperature (deg C)', fontsize=15)
axes.set_xlabel('Day of Year', fontsize=15)
axes.set_title('Record High and Low Temperatures', fontsize=20)

#Include 2015 data with record high and low temperatures (greater or less than 10 year temperature records) over the year

plt.scatter(max_record_2015.index.tolist(), max_record_2015['2015 Max Temp'].values, c = 'k', label = "2015 Record High")
plt.scatter(min_record_2015.index.tolist(), min_record_2015['2015 Min Temp'].values, c = 'r', label = "2015 Record Low")
plt.legend(['High Temperatures', 'Low Temperatures'])


#Fill between
axes.fill_between(XAxisDates, max_data_2005_2014['Data_Value'], min_data_2005_2014['Data_Value'], facecolor='grey', alpha=0.25)


#Dejunkify plot

axes.spines['top'].set_visible(False)
axes.spines['right'].set_visible(False)
plt.legend(loc = 8, fontsize=25)